In [316]:
import numpy as np
import matplotlib.pyplot as plt

#parameters
A = 1.0
M = 1.0
kappa = 1.0
dt = 0.000001
dx = 0.1
grid = 64
steps = 10000000

In [ ]:
init = []

#initial concentration field
for i in range(grid):
    if i < grid/2:
        init.append(0.0)
    else:
        init.append(1.0)

#initial concentration field as an array
#c = np.array(init,dtype=np.float64)

#c = np.zeros(grid)
#for i in range(grid):
#    if i <= grid/2-10:
#        c[i] = 0.0
#    elif i >= grid/2+10:
#        c[i] = 1.0
#    else:
#        c[i] = 0.0+1.0*(i+1.0-grid/2)/10.0
interface_width = 10.0
center = grid / 2
x = np.arange(grid)
c = np.clip((x - (center - interface_width / 2)) / interface_width, 0, 1)
print(c)

### The Laplacian function ###

What this does (in the space case, but can also be applied to the time case):
$$ \nabla^2 c_i^n= \frac{c^n_{i+1}-2c^n_i+c^n_{i-1}}{\Delta x^2}$$
but only for the interior points in the array. 
In order to enforce boundary conditions, the output is intialized as an array of zeros (in the same shape as the input), and then the finite difference method is used to approximate the second derivative of the interior points. The array is modified to include these approximations, and giving the output.

In [ ]:
def laplacian_g(u):
    u[0] = 0
    #create an array of zeros with the same shape as u
    lap = np.zeros_like(u)
    #apply the finite diff erence method to the interior points, in order to respect the boundary conditions
    lap[1:-1] = (u[:-2] - 2*u[1:-1] + u[2:]) / (dx*dx)
    return lap

def laplacian(u):
    lap = np.zeros_like(u)
    #print(len(lap))
    onebydx2 =  1.0 / (dx * dx)
    #print(onebydx2)
    for i in range(1, grid-1):
        lap[i] = (u[i-1] - 2*u[i] + u[i+1]) * onebydx2
    return lap

def laplacian_other(f): #5pt
    # Initialisation of derivative
    lap=np.zeros(grid) 
    # Calculation of 2nd derivative # Change the index to stay awway from the boundary  
    for i in range (2, grid-2):
        lap[i] = (-1./12 * f[i - 2] + 4./3  * f[i - 1] - 5./2 * f[i] \
                       +4./3  * f[i + 1] - 1./12  * f[i + 2]) / (dx * dx)
    return lap



### The for loop ###

At each time step, we want to update the concentration. The concentration, $c^{n+1}_i$, can be written, using the finite difference method, as

$$ c_i^{n+1}=c_i^n+\Delta t M \nabla^2 \mu^n_i$$

This is dependent on the Laplacian (second derivative) of the chemical potential at point $n$ ($\mu^n_i$), which is dependent on the chemical potentials at $n, n+1,$ and $ n-1$. The Laplacian of the chemical potential can be written as

$$ \nabla^2 \mu_i^n= \frac{\mu^n_{i+1}-2\mu^n_i+\mu^n_{i-1}}{\Delta x^2}$$

The chemical potential at point n, $\mu^n_i$
$$\mu^n_i=2Ac_i^n(1-c^n_i)(1-2c^n_i)-2\kappa\nabla^2c^n_i$$

This is dependent on the Laplacian of the concentration at point n, which can be rewritten as
$$ \nabla^2 c_i^n= \frac{c^n_{i+1}-2c^n_i+c^n_{i-1}}{\Delta x^2}$$

In summary, we have:

$$\mu^n_i=2Ac_i^n(1-c^n_i)(1-2c^n_i)-2\kappa\frac{c^n_{i+1}-2c^n_i+c^n_{i-1}}{\Delta x^2}$$

which is then used to find the updated concentration:

$$ c_i^{n+1}=c_i^n+\Delta t M \frac{\mu^n_{i+1}-2\mu^n_i+\mu^n_{i-1}}{\Delta x^2}$$





---

__Implementing this in code:__

__1)__ Find the Laplacian of the concentration (for every point in the grid):
$$ \nabla^2 c_i^n= \frac{c^n_{i+1}-2c^n_i+c^n_{i-1}}{\Delta x^2}$$

__2)__ Compute the chemical potential at each point:
$$\mu^n_i=2Ac_i^n(1-c^n_i)(1-2c^n_i)-2\kappa\nabla^2c^n_i$$

__3)__ Find the Laplacian of the chemical potential (for every point in the grid):
$$ \nabla^2 \mu_i^n= \frac{\mu^n_{i+1}-2\mu^n_i+\mu^n_{i-1}}{\Delta x^2}$$

__4)__ Update the concentration at each point:
$$ c_i^{n+1}=c_i^n+\Delta t M \nabla^2 \mu^n_i$$

In [ ]:

for time in range(steps):
    #step 1: calculate the laplacian of the concentration field
    lapc = laplacian(c)
    #step 2: compute the chemical potential at each point
    mu = 2.0 * A * c * (1.0 - c) * (1.0 - 2.0 * c) - 2.0 * kappa * lapc
    #step 3: find the laplacian of the chemical potential
    lap_mu = laplacian(mu)
    #step 4: update  the concentration fieldp 1: calculate the laplacian of the concentration field
    c += dt * M * lap_mu

plt.plot(c,'.-')
plt.show()

In [ ]:
print(c)
print(len(c))

### Finding the free energy ###

The free energy functional is given as:

$$\mathcal{F}=\int_{x} F dx$$

Where F, the free energy density, is made up of the chemical (bulk) and interfacial terms:

$$F=A(c_i^n)^2(1-c_i^n)^2 + \kappa|\nabla c_i^n|^2$$

Where the second term can be rewritten as:
$$\kappa|\nabla c_i^n|^2 = \kappa \nabla c_i^n \cdot \nabla c_i^n$$

Using the finite difference method, we can approximate $\nabla c^n_i$ as:
$$\nabla c_i^n = \frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}$$

To find the free energy of the system at each time step, we can approximate the integral by summing the free energy density at each grid point, then multiplying by the grid cell length.
$$\mathcal{F}_{sys} = \sum_{n=1}^{grid} \left(A(c_i^n)^2(1-c_i^n)^2 + \kappa|\nabla c_i^n|^2\right)\Delta x$$

---

### The gradient function ###

What this does (in the space case, but can also be applied to the time case):
$$\nabla c_i^n = \frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}$$
but only for the interior points in the array. 
In order to enforce boundary conditions, the output is intialized as an array of zeros (in the same shape as the input), and then the finite difference method is used to approximate the second derivative of the interior points. The array is modified to include these approximations, and giving the output.

In [321]:
def gradient(u):
    #create an array of zeros with the same shape as u
    grad = np.zeros_like(u)
    #apply the finite difference method to the interior points, in order to respect the boundary conditions
    grad[1:-1] = (u[2:] - u[:-2]) / (2*dx)
    return grad

Reinitializing the field:

In [322]:
init = []

#initial concentration field
for i in range(grid):
    if i < grid/2:
        init.append(0)
    else:
        init.append(1)

#initial concentration field as an array
c = np.array(init, dtype=float) 

__Implementing free energy calculation as an extension to the previous for loop:__

__5)__ Find the gradient of the concentration field using the finite difference method:

$$\nabla c_i^n = \frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}$$

__6)__ Find the free energy density at each point:
$$F=A(c_i^n)^2(1-c_i^n)^2 + \kappa\left(\frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}\cdot\frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}\right)$$

__7)__ Find the total free energy:
$$\mathcal{F}_{sys} = \sum_{n=1}^{grid} \left(A(c_i^n)^2(1-c_i^n)^2 + \kappa\left(\frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}\cdot\frac{c_i^{n+1}-c_i^{n-1}}{2\Delta x}\right)\right)\Delta x$$

In [ ]:
for time in range(steps):
    #step 1: calculate the laplacian of the concentration field
    lap = laplacian(c)
    #step 2: compute the chemical potential at each point
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * lap
    #step 3: find the laplacian of the chemical potential
    lap_mu = laplacian(mu)
    #step 4: update the concentration field
    c += dt * M * lap_mu
    #step 5: calculate the gradient of the concentration field
    grad_c = gradient(c)
    #step 6: calculate the free energy density
    interfacial = kappa * (grad_c**2)
    chemical =  A * c **2 * (1 - c) ** 2
    total = interfacial + chemical
    #step 7: calculate the total free energy
    free_energy = np.sum(total) * dx
    print(free_energy)


In [ ]:
#parameters
L = 10.0 #length of the domain
dx = 0.1 #cell size
x = np.arange(0, L, 0.1) #grid points

#initial condition
c = []
for i in range(len(x)):
    if x[i] < (L / 2 - 1.0):
        c.append(0.0)
    elif (L / 2 - 1.0) == x[i]:
        c.extend(np.arange(0.05, 1.05, 0.05))
    elif x[i] >= (L / 2 + 1.0):
        c.append(1.0)

c = np.array(c) #convert to numpy array

print(c)

#test = biharmonic(c, dx)

#print(test)

plt.plot(x, c, ".-", label='Initial Condition')
plt.legend()
plt.show()
